In [19]:
import numpy as np

In [20]:
train_x = np.load('train_x.npy')
train_y = np.load('train_y.npy')
test_x = np.load('test_x.npy')
test_y = np.load('test_y.npy')

In [21]:
np.shape(train_x)

(3730, 160, 160, 3)

In [22]:
np.shape(train_y)

(3730, 10)

In [23]:
np.shape(test_x)

(419, 160, 160, 3)

In [24]:
np.shape(test_y)

(419,)

In [25]:
train_x = train_x.astype('float32')
train_x /= 255

In [26]:
test_x = test_x.astype('float32')
test_x /= 255

In [30]:
from keras.applications.vgg16 import VGG16

In [28]:
model = VGG16(weights='imagenet', include_top=False)

In [29]:
outs_vgg16=[]
for i in range(0,3730):
    y=train_x[i]
    y=y.reshape(1,160,160,3)
    outs_vgg16.append(model.predict(y))

In [31]:
np.shape(outs_vgg16)

(3730, 1, 5, 5, 512)

In [33]:
outs_vgg16=np.array(outs_vgg16)

In [34]:
outs_vgg16=outs_vgg16.reshape([3730,5*5*512])

In [47]:
np.shape(outs_vgg16)

(3730, 12800)

In [ ]:
outs_test=[]
for i in range(0,419):
    y=test_x[i]
    y=y.reshape(1,160,160,3)
    outs_test.append(model.predict(y))

In [ ]:
outs_test=np.array(outs_test)
np.shape(outs_test)

In [ ]:
outs_test=outs_test.reshape([419,5*5*512])

In [78]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten,Activation
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Input
from keras.models import Model

In [79]:
model2=Sequential()
model2.add(Dense(1024,activation='relu',input_shape=(12800,)))
model2.add(Dropout(0.5))
model2.add(Dense(256,activation='relu'))
model2.add(Dropout(0.5))
model2.add(Dense(128,activation='relu'))
model2.add(Dropout(0.5))
model2.add(Dense(10, activation='softmax'))

In [80]:
model2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_18 (Dense)             (None, 1024)              13108224  
_________________________________________________________________
dropout_14 (Dropout)         (None, 1024)              0         
_________________________________________________________________
dense_19 (Dense)             (None, 256)               262400    
_________________________________________________________________
dropout_15 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_20 (Dense)             (None, 128)               32896     
_________________________________________________________________
dropout_16 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_21 (Dense)             (None, 10)                1290      
Total para

In [81]:
model2.compile(loss='categorical_crossentropy',
              optimizer='Adam',
              metrics=['accuracy'])

In [82]:
model2.fit(epochs=15,x=outs_vgg16,y=train_y,batch_size=64)

Epoch 1/15
3730/3730 [==============================] - 20s 5ms/step - loss: 2.7157 - acc: 0.1190
Epoch 2/15
3730/3730 [==============================] - 19s 5ms/step - loss: 2.2023 - acc: 0.1992
Epoch 3/15
3730/3730 [==============================] - 18s 5ms/step - loss: 1.8547 - acc: 0.3373
Epoch 4/15
3730/3730 [==============================] - 18s 5ms/step - loss: 1.6000 - acc: 0.4378
Epoch 5/15
3730/3730 [==============================] - 18s 5ms/step - loss: 1.4090 - acc: 0.5105
Epoch 6/15
3730/3730 [==============================] - 19s 5ms/step - loss: 1.2587 - acc: 0.5552
Epoch 7/15
3730/3730 [==============================] - 19s 5ms/step - loss: 1.1801 - acc: 0.5957
Epoch 8/15
3730/3730 [==============================] - 18s 5ms/step - loss: 1.1150 - acc: 0.6043
Epoch 9/15
3730/3730 [==============================] - 18s 5ms/step - loss: 1.0347 - acc: 0.6416
Epoch 10/15
3730/3730 [==============================] - 18s 5ms/step - loss: 0.9916 - acc: 0.6458
Epoch 11/15
3730/37

In [83]:
pred=model2.predict(x=outs_test,batch_size=64)

In [84]:
pred=np.argmax(a=pred,axis=1)

In [85]:
from sklearn.metrics import accuracy_score

In [86]:
accuracy_score(test_y, pred)

0.67780429594272074

In [87]:
model.save('vgg_model.h5')

In [88]:
model2.save('model.h5')